In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('241212_01_MLlib_regression').getOrCreate()

24/12/12 16:43:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
train_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/house_train.csv')

In [3]:
test_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/house_test.csv')

In [4]:
train_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [5]:
train_df.show(10)

24/12/12 16:44:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [6]:
# 결측치 0 
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [7]:
#encoding : 문자형 -> 숫자형 1,2,3,4

In [27]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder

In [28]:
string_columns = ['Neighborhood']
# 1,2,3,4 로 값을 단순화
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index")   for col in string_columns]

In [29]:
# onehotencoding : 범주형변수 1,2,3,4, -> 1로 바꾸는 인코딩
# 모두 1로 바꾼다.
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_encoded")   for col in string_columns]

## features selection

In [30]:
numeric_columns = [ "LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", 
    "1stFlrSF", "2ndFlrSF", "GrLivArea", "GarageCars", "GarageArea"]

In [31]:
assembler_inputs = [col+"_encoded" for col in string_columns] + numeric_columns

In [32]:
# assembler 
assembler = VectorAssembler(inputCols = assembler_inputs, outputCol = "features")

## Label Selection

In [33]:
train_df = train_df.withColumnRenamed("SalePrice","label")

## Pipelie setting

In [34]:
from pyspark.ml import Pipeline

In [35]:
pipeline = Pipeline(stages = indexers+encoders+[assembler])

In [36]:
pipeline_model = pipeline.fit( train_df )

In [37]:
train_transformed = pipeline_model.transform( train_df )

## 예측 -> 회귀 모델 학습 -> 평가 -> 예측

In [39]:
from pyspark.ml.regression import LinearRegression

In [41]:
lr = LinearRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_transformed)

24/12/12 17:47:21 WARN Instrumentation: [83f42166] regParam is zero, which might cause numerical instability and overfitting.
24/12/12 17:47:22 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/12 17:47:22 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/12/12 17:47:22 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/12/12 17:47:22 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [42]:
#fit : 모델 맞춤 과정 , 테스트 데이터에 의해 규칙이 변환됨
#따라서 test model은 fit 과정 없이 transform 과정 진행 

test_transformed = pipline_model.transform(test_df)
lr_model.transform(test_transformed)

NameError: name 'pipline_model' is not defined